In [1]:
import pandas as pd
import numpy as np
from __future__ import print_function
import csv
import pandas as pd
import math
from random import randrange
from math import sqrt
import copy
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [2]:
df = pd.read_csv("spam_data.csv", delim_whitespace=True, header=None)
X=df.drop([57],axis=1)


In [3]:
y=df[57]

In [4]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])


In [5]:

def class_counts(rows):
    counts = {} 
    for row in rows:
        
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts


In [6]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)


In [7]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


In [8]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity


In [9]:
def find_log(a):
    b = math.log(a, 2)
    return b


In [10]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)
    


In [11]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
     
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            self.column, condition, str(self.value))


In [12]:
class Leaf:
    

    def __init__(self, rows):
        self.predictions = class_counts(rows)


In [13]:
class Decision_Node:

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


In [14]:

def find_best_split(rows,features):
  
    best_gain = 0  
    best_question = None  
    current_uncertainty = gini(rows)

    for col in features: 

        values = set([row[col] for row in rows]) 

        for val in values:  

            question = Question(col, val)

           
            true_rows, false_rows = partition(rows, question)

           
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


In [15]:
def build_tree(rows,features):
    
    gain, question = find_best_split(rows,features)

    
    if gain == 0:
        return Leaf(rows)

  
    true_rows, false_rows = partition(rows, question)

   
    true_branch = build_tree(true_rows,features)

   
    false_branch = build_tree(false_rows, features)

   
    return Decision_Node(question, true_branch, false_branch)


In [16]:
def print_tree(node, spacing=""):
    

   
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    
    print(spacing + str(node.question))

   
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


In [17]:
def classify(row, node):

    if isinstance(node, Leaf):
        return node.predictions


    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


In [18]:
def subsample(dataset, ratio):
    
    sample = []
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample


In [19]:

def accuracy_metric(actual, predicted):
    
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


In [20]:
def test_train_split(data):
    K=10
    training_set = [x for i, x in enumerate(
        data) if i % K != 3 and i % K != 5 and i % K != 7]
    test_set = [x for i, x in enumerate(
        data) if i % K == 3 or i % K == 5 or i % K == 7]
    return training_set,test_set


In [21]:
def load_dataset():
    rows = []
    for line in open("spam_data.csv").readlines():
        line = line.strip().split(' ')
        features = [float(i) for i in line]
        rows.append(features)
    return rows

In [22]:
def predict_and_find_accuracy(tree,test_set):
    results=[]
    real_ans=[]
    for instance in test_set:
        result = tree.classify(instance[:-1])
        results.append(result)
        real_ans.append(float(instance[-1]))
    
    print(accuracy_metric(real_ans,results))


In [23]:
def predict(tree,row):
    return list(classify(row, tree).keys())[0]


In [24]:
def bagging_predict(trees, row):
     
    predictions = [predict(tree, row) for tree in trees]
   
    return max(set(predictions), key=predictions.count)


In [25]:
def get_features_list(n_features):
    features = []
    while len(features) < n_features:
        index = randrange(57)
        if index not in features:
            features.append(index)
    return features

    


In [26]:
def random_forest_PART_A(n_trees):
    data = load_dataset()
    train, test_data = test_train_split(data)
    trees = []
    n_features = 56
    features = get_features_list(n_features)
    for i in range(n_trees):
        
        print("Building the tree...")
        print("This may take a while, please wait...")
        my_tree = build_tree(train,features)

        trees.append(my_tree)
    
    predictions = [bagging_predict(trees, row) for row in test_data]
    real_ans=[]
    for instance in test_data:
        real_ans.append(instance[-1])
    print("accuracy of random forest from scratch with ",n_trees," trees= ",0.01*accuracy_metric(real_ans, predictions))


In [27]:
def random_forest_PART_A_SKLEARN_BUILD(n_estimators):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print("accuracy of random forest using sklearn with ",n_estimators," trees = ",
          accuracy_score(y_test, y_pred))


In [28]:
random_forest_PART_A_SKLEARN_BUILD(3)
random_forest_PART_A(3)


accuracy of random forest using sklearn with  3  trees =  0.944967414916727
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
accuracy of random forest from scratch with  3  trees=  0.9021739130434783


In [35]:
def random_forest_PART_B(n):
    data = load_dataset()
    train, test_data = test_train_split(data)
    trees = []
    n_trees=3
    n_features = randrange(n)
    features = get_features_list(n_features)
    print("number of fetures considered : ",n_features)
    for i in range(n_trees):
        print("Building the tree...")
        print("This may take a while, please wait...")
        my_tree = build_tree(train, features)

        trees.append(my_tree)
    predictions = [bagging_predict(trees, row) for row in test_data]
    real_ans = []
    for instance in test_data:
        real_ans.append(instance[-1])
    print("accuracy of random forest from scratch with ", n_trees,
          " trees= ", 0.01*accuracy_metric(real_ans, predictions))


In [36]:
# random_forest_PART_B()
random_forest_PART_B(20)
random_forest_PART_B(40)
random_forest_PART_B(50)


number of fetures considered :  16
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
accuracy of random forest from scratch with  3  trees=  0.803623188405797
number of fetures considered :  7
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
accuracy of random forest from scratch with  3  trees=  0.7130434782608696
number of fetures considered :  30
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
accuracy of random forest from scratch with  3  trees=  0.8384057971014492


In [34]:
def random_forest_PART_C(n_trees):
    data = load_dataset()
    train, test_data = test_train_split(data)
    trees = []
    n_features=randrange(57)
    features=get_features_list(n_features)
    for i in range(n_trees):
        sample = subsample(train, 1)

        
        print("Building the tree...")
        print("This may take a while, please wait...")
        my_tree = build_tree(sample,features)

        trees.append(my_tree)
    
    predictions = [bagging_predict(trees, row) for row in test_data]
    real_ans = []
    for instance in test_data:
        real_ans.append(instance[-1])
    print("out of bag error with ",n_trees," trees = ", 1-0.01*accuracy_metric(real_ans, predictions))


In [35]:
random_forest_PART_C(1)
random_forest_PART_C(3)
random_forest_PART_C(5)


Building the tree...
This may take a while, please wait...
out of bag error with  1  trees =  0.12463768115942031
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
out of bag error with  3  trees =  0.07608695652173902
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
Building the tree...
This may take a while, please wait...
out of bag error with  5  trees =  0.07246376811594202
